In [31]:
import os
import time
import pickle
import pandas as pd
import numpy as np
#import EDFlib

from scipy import signal
from scipy.signal import welch
from scipy.integrate import simps
from scipy.stats import f_oneway

#!pip install scikit-learn==0.20.3
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn import model_selection
from sklearn.metrics import classification_report, confusion_matrix
import itertools
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_recall_curve

import mne
from mne.preprocessing import (ICA, create_eog_epochs, create_ecg_epochs, corrmap)
# from mne.time_frequency import psd_welch
from mne.decoding import cross_val_multiscore


from fooof import FOOOFGroup
from fooof.bands import Bands
from fooof.analysis import get_band_peak_fg
from fooof.plts.spectra import plot_spectrum

import matplotlib.pyplot as plt
from matplotlib import cm
import seaborn as sns
%matplotlib inline

from pathlib import Path

In [32]:
current_dir  = Path.cwd()
data_dir = current_dir / 'User_data'

In [33]:
def csv_to_mne_raw(csv_file_path):
  #read the csv file into a pandas dataframe
  df = pd.read_csv(csv_file_path)

  #get the starting 10 columns corresponding to the 8 channels and the index data
  df1 = df.iloc[:,0:10]

  #also keep the timestamp, Cognitive Workload and Anxiety column
  df1['Timestamp'] = pd.to_datetime(df[' Timestamp (Formatted)'])
  df1['Cognitive Workload'] = df['Cognitive_Wokload']
  df1['Anxiety'] = df['Anxiety']

  #reindex the columns
  df1 = df1.reindex(columns=[' EXG Channel 0', ' EXG Channel 1', ' EXG Channel 2',
                            ' EXG Channel 3', ' EXG Channel 4', ' EXG Channel 5', 
                            ' EXG Channel 6', ' EXG Channel 7', 'Changed Sample Index',
                            'Sample Index','Cognitive Workload','Anxiety','Timestamp'])


  #drop the sample index column
  df1 = df1.drop(['Sample Index'], axis=1)

  #rename the channels
  df1 = df1.rename(columns={' EXG Channel 0': 'Fp1', ' EXG Channel 1': 'Fp2',
                            ' EXG Channel 2': 'C3', ' EXG Channel 3': 'C4',
                              ' EXG Channel 4': 'P3', ' EXG Channel 5': 'P4',
                              ' EXG Channel 6': 'O1', ' EXG Channel 7': 'O2',
                              'Changed Sample Index':'Index'})


  #convert dataframe to numpy array
  data = df1[['Fp1','Fp2','C3','C4','P3','P4','O1','O2']].to_numpy()
  # data = df1.to_numpy()

  #convert the data to float32
  data = data.astype('float32')

  # Scaling Data with the scale factor S = (4500000)/4/(2**23-1)
  data = data*4500000/4/(2**23-1)

  #get transpose of the data
  data = data.T

  # Create some metadata
  n_channels = 8
  sampling_freq = 250  # in Hertz
  ch_types = ['eeg'] * n_channels
  ch_names = ['Fp1','Fp2','C3','C4','P3','P4','O1','O2']
  info = mne.create_info( ch_names=ch_names, sfreq=sampling_freq,ch_types=ch_types)
  info.set_montage('standard_1020')
  # print(info)
  raw = mne.io.RawArray(data, info)
  raw.set_montage('standard_1020')
  return raw,df1.iloc[0]['Anxiety'] 

In [34]:
def createEDF(raw_object,new_file_path):
    #check if the file exists
    # if not os.path.isdir(new_file_path):
        # os.mkdir(new_file_path)
    mne.export.export_raw(str(new_file_path)+'.edf', raw_object, fmt='edf', overwrite=True)

    # if not os.path.isdir(new_file_path):
    #     os.mkdir(new_file_path)
    # mne.export.export_raw(new_file_path, raw_object, fmt='edf')
    

    

In [35]:
for user in os.listdir(data_dir):
    for session in os.listdir(data_dir / user):
        for file in os.listdir(data_dir/user/session):
            if file.endswith('.csv'):
                raw,_= csv_to_mne_raw(data_dir/user/session/file)
                createEDF(raw,data_dir/user/session/file[:-4])

[[-13488.25972667 -13417.666282   -13564.69850175 ... -13591.56879105
  -13669.3926758  -13567.37861561]
 [-18672.50622258 -18530.2213038  -18805.07860989 ... -18945.99518281
  -19102.50628844 -18910.2400751 ]
 [-22100.67359627 -22026.30305604 -22140.67204297 ... -21557.23703031
  -21639.1072793  -21565.45758283]
 ...
 [-20784.66435004 -20724.95166473 -20621.96739913 ... -20160.96057426
  -20201.73225163 -20258.54898905]
 [-15167.57632275 -15031.01205357 -15208.02634454 ... -15128.0807903
  -15268.2419441  -15146.40782377]
 [-15520.54459381 -15374.24998163 -15607.8463167  ... -15474.03340757
  -15626.69617144 -15463.97931351]]
Creating RawArray with float64 data, n_channels=8, n_times=46251
    Range : 0 ... 46250 =      0.000 ...   185.000 secs
Ready.
Overwriting existing file.
[[-14456.06581436 -14251.11987548 -14311.73152002 ... -13921.11906959
  -13749.50062943 -13756.69544643]
 [-19051.51706989 -18629.84442582 -18756.94762067 ... -19003.35045758
  -18638.15298833 -18646.44478703]


/tmp/ipykernel_733660/2964501602.py:5: RuntimeWarning: EDF format requires equal-length data blocks, so 0.996 seconds of zeros were appended to all channels when writing the final block.
  mne.export.export_raw(str(new_file_path)+'.edf', raw_object, fmt='edf', overwrite=True)


[[-15983.04444536 -16489.98724829 -16922.95555597 ... -16514.80711428
  -15887.41530067 -15702.61317299]
 [-22576.58973318 -23066.8053986  -23461.24942698 ... -22816.35619746
  -22125.07960574 -21985.38469535]
 [-22641.41118908 -22595.38405781 -22585.72600873 ... -22199.88181403
  -22238.17244776 -22221.69781511]
 ...
 [-23273.35228118 -23616.48229222 -23971.267341   ... -23288.87347223
  -22805.70908369 -22580.56485123]
 [-23106.60896604 -23053.89307523 -23041.50042991 ... -22767.88574112
  -22811.21180377 -22793.29967456]
 [-21583.26284276 -21592.34463593 -21602.3274838  ... -21220.52755787
  -21213.47837646 -21209.32304747]]
Creating RawArray with float64 data, n_channels=8, n_times=46251
    Range : 0 ... 46250 =      0.000 ...   185.000 secs
Ready.
Overwriting existing file.


/tmp/ipykernel_733660/2964501602.py:5: RuntimeWarning: EDF format requires equal-length data blocks, so 0.996 seconds of zeros were appended to all channels when writing the final block.
  mne.export.export_raw(str(new_file_path)+'.edf', raw_object, fmt='edf', overwrite=True)


[[-17225.47314582 -17080.31113844 -16652.86545788 ... -16206.51648793
  -16164.4477109  -16706.49811945]
 [-24209.64869562 -23943.26339917 -23569.34246316 ... -22968.11895437
  -22969.05353669 -23500.25880899]
 [-23348.22783114 -23372.90415702 -23394.41421859 ... -22881.96764783
  -22857.81309549 -22813.35338029]
 ...
 [-24523.91142869 -24412.88472605 -24155.70904681 ... -23497.18474563
  -23414.28771338 -23782.45447218]
 [-23574.12643497 -23603.3897579  -23630.51150432 ... -23314.23501877
  -23287.362634   -23235.29424529]
 [-22237.18338307 -22232.39941126 -22225.36699366 ... -21790.22253412
  -21787.79387738 -21792.66795466]]
Creating RawArray with float64 data, n_channels=8, n_times=46251
    Range : 0 ... 46250 =      0.000 ...   185.000 secs
Ready.
Overwriting existing file.


...

/tmp/ipykernel_733660/2964501602.py:5: RuntimeWarning: EDF format requires equal-length data blocks, so 0.996 seconds of zeros were appended to all channels when writing the final block.
  mne.export.export_raw(str(new_file_path)+'.edf', raw_object, fmt='edf', overwrite=True)


[[-22539.1079532  -22685.09767362 -22702.11084361 ... -22105.47433188
  -22213.35153404 -22328.4434602 ]
 [-23551.14115825 -23700.20494203 -23718.65770406 ... -23188.02868767
  -23297.47121043 -23414.97083857]
 [-23427.61494101 -23568.51475012 -23582.81427864 ... -22972.59698668
  -23076.15750818 -23185.09711669]
 ...
 [-25147.92114397 -25289.97136943 -25307.94007649 ... -24618.64161416
  -24723.46151676 -24836.62770045]
 [-24916.82366408 -25055.38492192 -25073.94664871 ... -24423.71205076
  -24525.33216145 -24634.59447327]
 [-19492.93748019 -19635.06104731 -19654.62860259 ... -19055.33083629
  -19159.73583464 -19272.88315904]]
Creating RawArray with float64 data, n_channels=8, n_times=46251
    Range : 0 ... 46250 =      0.000 ...   185.000 secs
Ready.
Overwriting existing file.


/tmp/ipykernel_733660/2964501602.py:5: RuntimeWarning: EDF format requires equal-length data blocks, so 0.996 seconds of zeros were appended to all channels when writing the final block.
  mne.export.export_raw(str(new_file_path)+'.edf', raw_object, fmt='edf', overwrite=True)


[[-23722.59719902 -23798.53305985 -23789.55813596 ... -23165.24038571
  -23180.47449654 -23103.53280721]
 [-24666.98634194 -24747.11315486 -24738.85697925 ... -24191.1770743
  -24207.47149601 -24128.04666757]
 [-24710.81741432 -24781.8247155  -24771.46468195 ... -24088.78373295
  -24101.06950899 -24026.48313473]
 ...
 [-26259.41402391 -26337.65281292 -26330.0797625  ... -25628.40751413
  -25645.09798081 -25567.86711578]
 [-26094.12706462 -26168.83916744 -26162.29290028 ... -25504.95463855
  -25521.06884932 -25446.66268599]
 [-20642.16569271 -20720.63707955 -20713.09965223 ... -20063.00964213
  -20079.19719456 -20001.22872196]]
Creating RawArray with float64 data, n_channels=8, n_times=46251
    Range : 0 ... 46250 =      0.000 ...   185.000 secs
Ready.
Overwriting existing file.


/tmp/ipykernel_733660/2964501602.py:5: RuntimeWarning: EDF format requires equal-length data blocks, so 0.996 seconds of zeros were appended to all channels when writing the final block.
  mne.export.export_raw(str(new_file_path)+'.edf', raw_object, fmt='edf', overwrite=True)
/tmp/ipykernel_733660/2964501602.py:5: RuntimeWarning: EDF format requires equal-length data blocks, so 0.996 seconds of zeros were appended to all channels when writing the final block.
  mne.export.export_raw(str(new_file_path)+'.edf', raw_object, fmt='edf', overwrite=True)


In [30]:
import eegraph

# curr_file = data_dir / '12349/session_1/12349_session_1_23-03-2023_23-43.edf'
G = eegraph.Graph()
G.load_data(path = "User_data/12349/session_1/12349_session_1_23-03-2023_23-43.edf")

Extracting EDF parameters from /home/niskal/Abhishek/OpenBCI_DataAnlaysis/User_data/12349/session_1/12349_session_1_23-03-2023_23-43.edf...
EDF file detected


ValueError: could not convert string to float: '-J374935'

In [ ]:
import eegraph

curr_file = data_dir / '12349/session_1/12349_session_1_23-03-2023_23-43.edf'
G = eegraph.Graph()
G.load_data(path = str(curr_file))